# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable, List
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__

    
def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [2]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [3]:
# также нужно реализовать __sub__ и __rsub__

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    
            
    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(self.value - other, self.d)
            
    def __rsub__(self, other: Number) -> "Dual":
        match other:
            case Number():
                return Dual(-self.value + other, -self.d)
    
    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)
    
    def __pos__(self) -> "Dual":
        return Dual(self.value, self.d)
    
    def __abs__(self) -> "Dual":
        return Dual(abs(self.value), float(bool(self.value > 0) - bool(self.value < 0))) # -1 when self.value < 0
                                                                                         #  1 when self.value > 0
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / float(o_value), 
                            (self.d * o_value - o_d * self.value) / (o_value * o_value))
            case Number():
                return Dual(self.value / float(other), self.d / float(other))
            
    def __rtruediv__(self, other: Number) -> "Dual":
        match other:
            case Number():
                return Dual(float(other) / self.value, -float(other) * self.d / (self.value * self.value))
            
    def __pow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual():
                raise TypeError("unsupported")
            case Number():
                return Dual(pow(self.value, other), float(self.d * other * pow(self.value, other - 1)))
            
    def __rpow__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual():
                raise TypeError("unsupported")
            case Number():
                return Dual(pow(other, self.value), math.log(float(other), math.e) * pow(float(other), self.value))
                
    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__

In [4]:
import numpy as np

f_neg = lambda x: 2 * (-x)
f_neg_diff = diff(f_neg)
assert np.isclose(f_neg_diff(1), -2), "__neg__ method implemented wrong"

f_abs = lambda x: abs(x)
f_abs_diff = diff(f_abs)
assert np.isclose(f_abs_diff(2), 1), "__abs__ method implemented wrong"

f_truediv1 = lambda x: x / 10
f_truediv1_diff = diff(f_truediv1)
assert np.isclose(f_truediv1_diff(1), 1/10), "__truediv__ method implemented wrong"

f_truediv2 = lambda x: (2 * x) / (x ** 2)
f_truediv2_diff = diff(f_truediv2)
assert np.isclose(f_truediv2_diff(1), -2), "__truediv__ method implemented wrong"

f_rtruediv = lambda x: 10 / x
f_rtruediv_diff = diff(f_rtruediv)
assert np.isclose(f_rtruediv_diff(1), -10), "__rtruediv__ method implemented wrong"

f_pow = lambda x: x ** 2
f_pow_diff = diff(f_pow)
assert np.isclose(f_pow_diff(1), 2), "__pow__ method implemented wrong"

f_general = lambda x: (-x + 1)**2 + (2 * x) / (3 * x ** 3) + (1 / x) + x / 10 + 10 * abs(x)
f_general_diff = diff(f_general)
assert np.isclose(f_general_diff(2), 701 / 60), "something went wrong"

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [5]:
import math
            
def exp(x: Union["Dual", Number]) -> "Dual":
    match x:
        case Dual():
            return Dual(math.exp(x.value), x.d * math.exp(x.value))
        case Number():
            return Dual(math.exp(x), 0)
    return Dual(self.value, self.d)

def cos(x: Union["Dual", Number]) -> "Dual":
    match x:
        case Dual():
            return Dual(math.cos(x.value), -x.d * math.sin(x.value))
        case Number():
            return Dual(math.cos(x), 0)

def sin(x: Union["Dual", Number]) -> "Dual":
    match x:
        case Dual():
            return Dual(math.sin(x.value), x.d * math.cos(x.value))
        case Number():
            return Dual(math.sin(x), 0)
        
def log(x: Union["Dual", Number], base: float = math.e) -> "Dual":
    match x:
        case Dual():
            return Dual(math.log(x.value, base), x.d / (x.value * math.log(base)))
        case Number():
            return Dual(math.log(x, base), 0)

In [6]:
f_exp = lambda x: exp(2) * x + exp(x)
f_exp_diff = diff(f_exp)
assert np.isclose(f_exp_diff(2), math.exp(2) + math.exp(2)), "exp implemented wrong"

f_cos = lambda x: cos(0) * x + cos(x)
f_cos_diff = diff(f_cos)
assert np.isclose(f_cos_diff(math.pi/2), 1 - math.sin(math.pi/2)), "cos implemented wrong"

f_sin = lambda x: sin(0) * x + sin(x)
f_sin_diff = diff(f_sin)
assert np.isclose(f_sin_diff(0), 0 + math.cos(0)), "sin implemented wrong"

f_log = lambda x: log(8, 2) * x + log(x, math.e)
f_log_diff = diff(f_log)
assert np.isclose(f_log_diff(2), math.log(8, 2) + 1 / 2), "log implemented wrong"

f_general2 = lambda x: 2 * exp(x) / x - log(x) + 1 / cos(x) + sin(x) * cos(x)
f_general2_diff = diff(f_general2)
assert np.isclose(f_general2_diff(1),
                  -1 + math.sin(1) / pow(math.cos(1), 2) + pow(math.cos(1), 2) - pow(math.sin(1), 2)), "something went wrong"

f_comp = lambda x: exp(cos(x)) + 2 * log(sin(x))
f_comp_diff = diff(f_comp)
assert np.isclose(f_comp_diff(1), 2 / math.tan(1) - pow(math.e, math.cos(1)) * math.sin(1)), "composition is wrong"

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [7]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [8]:
def my_derivative(func: Callable[[float], float], value: float) -> float:
    dx = 1e-8
    f_left = func(Dual(value + dx, 1.0)).value
    f_right = func(Dual(value - dx, 1.0)).value
    return (f_left - f_right) / (2 * dx)

In [9]:
f_check1 = lambda x: 5 * x ** 2 + 2 * x + 2
f_check1_diff = diff(f_check1)
assert np.isclose(f_check1_diff(10), my_derivative(f_check1, 10)), "not equal"

f_check2 = lambda x: exp(x ** 2) + cos(sin(x))
f_check2_diff = diff(f_check2)
assert np.isclose(f_check2_diff(5), my_derivative(f_check2, 5)), "not equal"

f_check3 = lambda x: 2 / exp(pow(cos(x), 2)) - log(x**2 + 2*x, 2)
f_check3_diff = diff(f_check3)
assert np.isclose(f_check3_diff(5), my_derivative(f_check3, 5)), "not equal"

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [10]:
# функция для генерации полиномов 1-3 степени
def generate_polynomial() -> str:
    degree = np.random.randint(1, 4) # degree from 1 to 3
    coeffs = []
    bias = np.random.uniform(-5, 5)
    str_func = f"{bias:.2f} "
    for i in range(1, degree + 1):
        coef = np.random.uniform(-20, 20)
        if coef < 0:
            str_func += f"- {abs(coef):.2f}*x**{i} "
        else:
            str_func += f"+ {coef:.2f}*x**{i} "
    return str_func[:-1].replace("x**1", 'x')

# функция для генерации одночленов
def generate_monomial() -> str:
    # decide whether to include bias with prob. 0.2
    bool_bias = bool(np.random.binomial(1, 0.2))
    if bool_bias:
        bias = np.random.uniform(-5, 5)
    coef = 0
    while coef == 0:
        coef = np.random.randint(-2, 3)
    if bool_bias:
        if bias < 0:
            return f"{'-'*(coef == -1) if abs(coef) == 1 else str(coef)+'*'}x - {abs(bias):.2f}"
        else:
            return f"{'-'*(coef == -1) if abs(coef) == 1 else str(coef)+'*'}x + {bias:.2f}"
    return f"{'-'*(coef == -1) if abs(coef) == 1 else str(coef)+'*'}x"

# выдает случайную функцию с распределением, смещенным в сторону одночленов и полиномов
# используется для первоначальной генерации функции для одного из слагаемых
def get_outside_function() -> str:
    func_set = {0: "cos", 1: "sin", 2: "log", 3: "exp", 4: "poly", 5: "mono"}
    choice = np.random.choice(np.arange(6), p=[0.1, 0.1, 0.1, 0.1, 0.2, 0.4])
    return func_set[choice]

# используется для генерации функции, если на предыдущем шаге была сгенерирована cos, sin, log, exp
# распределение сильно смещенно в сторону одночленов и полиномов 
def get_inside_function() -> str:
    func_set = {0: "cos", 1: "sin", 2: "log", 3: "exp", 4: "poly", 5: "mono"}
    choice = np.random.choice(np.arange(6), p=[0.05, 0.05, 0.05, 0.05, 0.15, 0.65])
    return func_set[choice]

# генерирует случайную бинарную операцию с распределением, смещенным в сторону + -
def get_operation() -> str:
    operations_set = {0: "+", 1: "-", 2: "*", 3: "/"}
    choice = np.random.choice(np.arange(4), p=[0.4, 0.4, 0.1, 0.1])
    return operations_set[choice]

# проверяет, является ли сгенерированная функция полиномом
def check_if_poly(func_to_check: str) -> bool:
    return func_to_check in {"mono", "poly"}

# генерирует либо одночлен, либо полином
def generate_poly_or_mono(f_str: str) -> str:
    if f_str == "mono":
        return generate_monomial()
    else:
        return generate_polynomial()

# функция для генерации случайной функции с несколькими слагаемыми
def generate_elementary() -> str:
    s = ""
    terms = 5
    for i in range(terms):
        f = get_outside_function()
        is_poly = check_if_poly(f)
        brackets_cnt = 0
        if is_poly:
            poly = generate_poly_or_mono(f)
            s += f"({poly})"
        while not is_poly:
            s += f"{f}("
            brackets_cnt += 1
            f = get_inside_function()
            is_poly = check_if_poly(f)
            if is_poly:
                poly = generate_poly_or_mono(f)
                s += poly
        s += ")" * brackets_cnt
        op = get_operation()
        if i != terms - 1:
            s += f" {op} "                 
    return s

In [11]:
num_tries = 10
for i in range(num_tries):
    flag = False
    while not flag:
        generated_f = generate_elementary()
        func = eval(f"lambda x: {generated_f}")
        func_diff = diff(func)
        value = np.random.normal(loc=0, scale=3)
        try:
            assert np.isclose(func_diff(value), my_derivative(func, value)), f"error for f(x) = {generated_f} at x = {value:.2f}"
            flag = True
        except (ValueError, OverflowError):
            continue
    print(f"All is good for f(x) = {generated_f} at x = {value:.2f}")    
    print('-'*100)

All is good for f(x) = log(2*x) + (2*x) - (x) - (x) - (0.38 + 19.69*x) at x = 2.15
----------------------------------------------------------------------------------------------------
All is good for f(x) = (x) + (2*x) - (3.52 + 10.57*x) - (-1.14 - 8.53*x - 8.43*x**2 + 0.75*x**3) - exp(2*x) at x = 3.25
----------------------------------------------------------------------------------------------------
All is good for f(x) = (4.18 - 8.96*x) + (2*x) + (2*x) * cos(x - 0.01) + exp(x) at x = -0.36
----------------------------------------------------------------------------------------------------
All is good for f(x) = (-2*x) + (-3.39 - 15.83*x + 2.50*x**2 - 10.46*x**3) + (-3.13 + 3.43*x + 0.09*x**2 + 19.13*x**3) * log(exp(-2*x)) - (-2.73 - 18.43*x) at x = -2.10
----------------------------------------------------------------------------------------------------
All is good for f(x) = exp(x) * (-2.09 - 2.42*x + 0.77*x**2) - (1.52 + 13.33*x + 13.05*x**2 - 17.47*x**3) / exp(-2*x) / (x) at x = 

Среди многих запусков выяснилось, что проблемы возникают тогда, когда значение производной слишком большое, и функция `my_derivative()` не справляется с подсчетом по определению. Например при $f(x) = exp(-1*x + 1.03) - sin(exp(0.38 - 11.71*x + 11.46*x^2 - 1.74*x^3)) - sin(exp(4.53 - 12.75*x + 9.68*x^2))$ в точке $x = -1.87$ функция `func_diff()` выдает $-2.674316437895079e+33$, а `my_derivative()` выдает $103236211.9$

Отмечу, что автоматическое дифференцирование `func_diff()` всё еще выдает нужное значение (проверено вручную с Wolfram).

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [12]:
def gradient(func: Callable[List[float], float]) -> Callable[List[float], float]:
    # нужно вернуть
    # [func(Dual(var[0], 1.), Dual(var[1], 0), ..., Dual(var[n-1], 0)).d,
    #  func(Dual(var[0], 0.), Dual(var[1], 1.), ..., Dual(var[n-1], 0)).d,
    #  ...,
    #  func(Dual(var[0], 0), Dual(var[1], 0), ..., Dual(var[n-1], 1.)).d]
    def take_gradient(*VARS):
        gradient_vector = []
        for i in range(len(VARS)):
            partial_derivative_i = func(*[Dual(VARS[i], float(i == j)) for j in range(len(VARS))]).d
            gradient_vector.append(partial_derivative_i)
        return gradient_vector[0] if len(VARS) == 1 else gradient_vector
    return take_gradient

In [13]:
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  

f_diff = gradient(f)
f_diff(10, 10, 10) # = [10, 5, 1]

[10.0, 5.0, 1.0]

In [14]:
def g(x: float, y: float, z: float) -> float:
    return exp(2*x + y) - log(z / 10)

g_diff = gradient(g)
assert (np.asarray(g_diff(1, 1, 10)) == np.array([2*math.exp(2*1 + 1), math.exp(2*1 + 1), - 1 / 10])).all(), "error"